In [9]:
import re
import sys
import json
import numpy as np
from operator import itemgetter
from itertools import chain
#############################################
# Settings
#############################################

In [3]:
nb_infile = "book/content/chapter2_1.ipynb"
nb_outfile = "book/content/chapter2_1_new.ipynb"
bib_filename = None

In [4]:
# Load Jupyter Notebook as Dictionary
try:
    with open(nb_infile,'r') as fp:
        nb = json.load(fp)
    fp.close()
    print("Input Jupyter Notebook read successfully")
except:
    print("Couldn't find Jupyter Notebook. Check your input path")

Input Jupyter Notebook read successfully


In [36]:
def split_cell_content(cell_content, latex_markers_positions):
    first_idx = latex_markers_positions[0]
    last_idx = latex_markers_positions[-1]
    latex_markers_positions = [0] + latex_markers_positions + [-1]
    
    eqs_begin_indices = latex_markers_positions[1:-1:2]
    eqs_end_indices = list(np.array( latex_markers_positions[2::2] ) + 1)
    eqs_idx_pairs = list(zip(eqs_begin_indices, eqs_end_indices))
    
    text_begin_indices = list(np.array( latex_markers_positions[2::2] ) + 1)
    text_end_indices = list(np.array( latex_markers_positions[3:-1:2] ) + 0)
    text_idx_pairs = [(0,first_idx)] + list(zip(text_begin_indices, text_end_indices)) + [(last_idx+1, -1)]
    
    all_idx_pairs = [(x, 'txt') for x in text_idx_pairs] + [(x, 'eq') for x in eqs_idx_pairs]
    all_idx_pairs.sort(key = lambda x: x[0][0])

    cell_content_split = [cell_content[idx_pair[0][0]: idx_pair[0][1]] for idx_pair in all_idx_pairs]
    cell_content_type = [idx_pair[1] for idx_pair in all_idx_pairs]
    
    return cell_content_split, cell_content_type

In [48]:
def insert_math_env(cell_content_split, cell_content_type):
    cell_content_new = []
    N = len(cell_content_split)
    for i in range(N):
        content_chunk = cell_content_split[i]
        content_type = cell_content_type[i]
        if content_type == "eq":
            label_name = ""
            for line in content_chunk:
                # Find all LaTeX equation labels
                item_matched = re.findall(latex_eqlabel_pattern, line)
                if len(item_matched) > 0:
                    label_type, label_name = item_matched[0].strip().split(":")

            upper = ["```{math}\n", "---\n", "label: "+label_name+"\n", "---\n"]
            body  =  content_chunk
            lower = ["```"]
            content_chunk =  list(chain.from_iterable([upper, body, lower])) 
            cell_content_new.append(content_chunk)
        else:
            cell_content_new.append(content_chunk)

    return list(chain.from_iterable(cell_content_new))

In [37]:
markdown_env_equation = ["```{math}", "```"] # MyST Markdown Equation environment markers
markdown_eqlabel = ["---", "label: ", "---"] # MyST Markdown equation label format

latex_eqlabel_pattern = re.compile(r'\\label\{(.+?)\}') # LaTeX equation label pattern
latex_env_equation = [r"\\begin{equation}", r"\\end{equation}"] #LaTeX equation environment markers
latex_env_eqnarray = [r"\\begin{eqnarray}", r"\\end{eqnarray}"] #LaTeX eqnarray environment markers

In [52]:
PATTERNS = latex_env_equation + latex_env_eqnarray

for i in range(len(nb['cells'])):
    cell = nb['cells'][i]
    latex_markers_positions = []
    cell_content = cell['source']
    
    for j in range(len(cell['source'])):
        item = cell['source'][j]

        for string in PATTERNS:
            item_matched = re.findall(string, item)
            if len(item_matched) > 0:
                latex_markers_positions.append(j)
    if len(latex_markers_positions) > 0:
        cell_content_split, cell_content_type = split_cell_content(cell_content, latex_markers_positions)
        cell_content_modified = insert_math_env(cell_content_split, cell_content_type)
        nb['cells'][i]['source'] = cell_content_modified

In [54]:
with open(nb_outfile,'w') as fp:
    json.dump(nb, fp)

Modify cell content when equations present

In [41]:
len(cell_content_split), len(cell_content_type)

(17, 17)

In [48]:
def insert_math_env(cell_content_split, cell_content_type):
    cell_content_new = []
    N = len(cell_content_split)
    for i in range(N):
        content_chunk = cell_content_split[i]
        content_type = cell_content_type[i]
        if content_type == "eq":
            label_name = ""
            for line in content_chunk:
                # Find all LaTeX equation labels
                item_matched = re.findall(latex_eqlabel_pattern, line)
                if len(item_matched) > 0:
                    label_type, label_name = item_matched[0].strip().split(":")

            upper = ["```{math}\n", "---\n", "label: "+label_name+"\n", "---\n"]
            body  =  content_chunk
            lower = ["```"]
            content_chunk =  list(chain.from_iterable([upper, body, lower])) 
            cell_content_new.append(content_chunk)
        else:
            cell_content_new.append(content_chunk)

    return list(chain.from_iterable(cell_content_new))

In [49]:
insert_math_env(cell_content_split, cell_content_type)

['The NHIM and its stable and unstable manifolds can be found analogously to the  2 degree of freedom system case. If $q_1=p_1=0$ then $\\dot{q_1}=0$ and $\\dot{p_1}=0$ and the system consists of $n-1$ harmonic oscillators bounded by the fixed total energy. For $E=0$, $q_1,p_1=0$ is an unstable equilibrium point and for $E>0$, an invariant set that can be shown to be a sphere. The dimension of the sphere depends on the number of degrees of freedom with non-zero energy. This sphere is a manifold, it is invariant and normally hyperbolic, hence a NHIM. In general a NHIM does not have to be a sphere. If all degrees of freedom have non-zero energy, except for $q_1,p_1=0$, the NHIM is $2n-3$ dimensional and its stable and unstable manifolds are dynamical barriers on the energy level set according to Tab. \\ref{tab:dimensions}.\n',
 '\n',
 '```{math}\n',
 '---\n',
 'label: \n',
 '---\n',
 '\\begin{equation}\n',
 '\\displaystyle{ \\Gamma = \\lbrace  (0,q_2,...,q_n, 0,p_2,...,p_n ) | \\sum_{i=2

In [12]:
latex_markers_positions = [2, 5, 16, 20, 26, 29, 33, 35, 39, 42, 46, 49, 53, 55]
cell_content = nb['cells'][6]['source']
split_cell_content(cell_content, latex_markers_positions)

[['The equations of motion are \n', '\n'],
 ['\\begin{eqnarray}\n',
  '\\dot{q_1} &=& \\frac{\\partial H}{ \\partial p_1} =\\lambda p_1  \\nonumber \\\\\n',
  '\\dot{p_1} &=& -\\frac{\\partial H}{ \\partial q_1} = \\lambda q_1.\n',
  '\\end{eqnarray}\n'],
 ['\n',
  'For energies $E_1<0$ the energy level set consists of two disconnected parts, a trajectory cannot reach $q_1=0$ due insufficient energy. For $E_1=0$ the two previously disconnected parts connect at $q_1=p_1=0$ and for energies $E_1>0$, any value of $q_1$ is accessible.\n',
  '\n',
  'The origin, $\\mathcal{M} = (q_1=0,p_1=0)$, is a equilibrium point of the system. The trajectory that starts at $\\mathcal{M}$ remains always at the same point.\n',
  '\n',
  '\n',
  '\\textbf{Showing the invariance of candidate sets.}\n',
  '\n',
  'Let us consider the line $ q_1 = - p_1$. The Hamiltonian vector field evaluated on the line is tangent to the line.\n',
  '\n'],
 ['\\begin{eqnarray}\n',
  '\\dot{q_1} &=& -\\lambda q_1  \\nonumber

In [14]:
latex_markers_positions = [2, 5, 16, 20, 26, 29, 33, 35, 39, 42, 46, 49, 53, 55]
first_idx = latex_markers_positions[0]
last_idx = latex_markers_positions[-1]

latex_markers_positions = [0] + latex_markers_positions + [-1]

In [15]:
import numpy as np
from operator import itemgetter

In [16]:
eqs_begin_indices = latex_markers_positions[1:-1:2]
eqs_end_indices = list(np.array( latex_markers_positions[2::2] ) + 1)

eqs_range = list(zip(eqs_begin_indices, eqs_end_indices))

In [17]:
print(eqs_range)

[(2, 6), (16, 21), (26, 30), (33, 36), (39, 43), (46, 50), (53, 56)]


In [18]:
text_begin_indices = list(np.array( latex_markers_positions[2::2] ) + 1)
text_end_indices = list(np.array( latex_markers_positions[3:-1:2] ) + 0)

text_range = [(0,first_idx)] + list(zip(text_begin_indices, text_end_indices)) + [(last_idx+1, -1)]

In [19]:
print(text_range)

[(0, 2), (6, 16), (21, 26), (30, 33), (36, 39), (43, 46), (50, 53), (56, -1)]


In [23]:
y = [(x, 'txt') for x in text_range] + [(x, 'eq') for x in eqs_range]

In [25]:
y.sort(key = lambda x: x[0][0])

In [26]:
y

[((0, 2), 'txt'),
 ((2, 6), 'eq'),
 ((6, 16), 'txt'),
 ((16, 21), 'eq'),
 ((21, 26), 'txt'),
 ((26, 30), 'eq'),
 ((30, 33), 'txt'),
 ((33, 36), 'eq'),
 ((36, 39), 'txt'),
 ((39, 43), 'eq'),
 ((43, 46), 'txt'),
 ((46, 50), 'eq'),
 ((50, 53), 'txt'),
 ((53, 56), 'eq'),
 ((56, -1), 'txt')]

In [20]:
y = text_range + eqs_range
y.sort(key = lambda x: x[0])
y

[(0, 2),
 (2, 6),
 (6, 16),
 (16, 21),
 (21, 26),
 (26, 30),
 (30, 33),
 (33, 36),
 (36, 39),
 (39, 43),
 (43, 46),
 (46, 50),
 (50, 53),
 (53, 56),
 (56, -1)]

In [27]:
cell_content = nb['cells'][6]['source']
for x in y:
    init, end = x[0]
    xtype = x[1]
    print(xtype, cell_content[init: end])

txt ['The equations of motion are \n', '\n']
eq ['\\begin{eqnarray}\n', '\\dot{q_1} &=& \\frac{\\partial H}{ \\partial p_1} =\\lambda p_1  \\nonumber \\\\\n', '\\dot{p_1} &=& -\\frac{\\partial H}{ \\partial q_1} = \\lambda q_1.\n', '\\end{eqnarray}\n']
txt ['\n', 'For energies $E_1<0$ the energy level set consists of two disconnected parts, a trajectory cannot reach $q_1=0$ due insufficient energy. For $E_1=0$ the two previously disconnected parts connect at $q_1=p_1=0$ and for energies $E_1>0$, any value of $q_1$ is accessible.\n', '\n', 'The origin, $\\mathcal{M} = (q_1=0,p_1=0)$, is a equilibrium point of the system. The trajectory that starts at $\\mathcal{M}$ remains always at the same point.\n', '\n', '\n', '\\textbf{Showing the invariance of candidate sets.}\n', '\n', 'Let us consider the line $ q_1 = - p_1$. The Hamiltonian vector field evaluated on the line is tangent to the line.\n', '\n']
eq ['\\begin{eqnarray}\n', '\\dot{q_1} &=& -\\lambda q_1  \\nonumber \\\\\n', '\\dot{p_

In [ ]:

def replace_citation_syntax(citations_matched_raw, item):
    item_modified = item
    for match in citations_matched_raw:
        bib_tags_raw = re.split(",", match)
        bib_tags_clean = [x.strip() for x in bib_tags_raw]
        #############################################
        # Replace LaTeX syntax with jekyll's in cell items
        #############################################
#         jekyll_syntax_items = ["{% cite"]+bib_tags_clean+['--file '+bib_filename]+["%}"]
        jekyll_syntax_items = ["{cite}`"]+[",".join(bib_tags_clean)]+["`"]
        separator = ''
        chunk_new = separator.join(jekyll_syntax_items)
        chunk_original = '\cite{'+match+'}'
        item_modified = item_modified.replace(chunk_original, chunk_new)
    return item_modified

def replace_bibliography_syntax(bibliography_matched_raw, item):
    # Only one single bibliography line exists
    bib_tags_raw = re.split(",", bibliography_matched_raw[0])
    bib_tags_clean = [x.strip() for x in bib_tags_raw]
    separator = ' '
    jekyll_syntax_items = ["{% bibliography"]+[separator.join(['--file', x]) for x in bib_tags_clean]+["--cited %}"]
    chunk_new = separator.join(jekyll_syntax_items)
    chunk_original = r'\bibliography{'+bibliography_matched_raw[0]+'}'
    item_modified = item.replace(chunk_original, chunk_new)
    return item_modified

def latex_to_jekyll_bib(nb):
    for i in range(len(nb['cells'])):
        cell = nb['cells'][i]
        for j in range(len(cell['source'])):
            item = cell['source'][j]

            citations_matched_raw = re.findall(citations_pattern,item)
            bibliography_matched_raw = re.findall(bibliography_pattern,item)

            if citations_matched_raw:
                item_modified = replace_citation_syntax(citations_matched_raw, item)
                nb['cells'][i]['source'][j] = item_modified

            elif bibliography_matched_raw:
                item_modified = replace_bibliography_syntax(bibliography_matched_raw, item)
                nb['cells'][i]['source'][j] = item_modified
                
            else:
                pass
    #############################################
    # Modify cell content with new syntax accordingly
    #############################################
    return nb
    
